<font size="5"> Solution to 6 </font>

Dataset used in the question is available at http://www.dt.fee.unicamp.br/~tiago/smsspamcollection and is redacted from this repository as requested in the question.

In [9]:
import collections

import numpy as np

import util
import svm


def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """
    return message.lower().split()


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message. 

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """
    dictionary = dict()

    for message in messages:
        for word in get_words(message):
            dictionary[word] = dictionary.get(word, 0) + 1

    # Remove words which appear less than five times
    if dictionary[word] < 5:
        del dictionary[word]

    # Sort the dictionary
    dictionary = dict(sorted(dictionary.items()))
    
    return dictionary


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each 
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that 
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    m = len(messages)
    n = len(word_dictionary)

    # Create a numpy array of zeros
    array = np.zeros((m, n), dtype=int)

    for i, message in enumerate(messages):
        for word in get_words(message):
            if word in word_dictionary:
                array[i, word_dictionary[word]] += 1
    
    return array


def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # Maximum likelihood estimates for parameters
    p_y = np.mean(labels)

    spam = matrix[labels == 0]
    p_xj_y0 = (np.sum(spam, axis=0) + 1) / (np.sum(spam) + 2)

    not_spam = matrix[labels == 1]
    p_xj_y1 = (np.sum(not_spam, axis=0) + 1) / (np.sum(not_spam) + 2)

    # Store the parameters in a dictionary
    model = { 'p_y': p_y, 'p_xj_y0': p_xj_y0, 'p_xj_y1': p_xj_y1 }

    return model

def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    p_y = model['p_y']
    p_xj_y0 = model['p_xj_y0']
    p_xj_y1 = model['p_xj_y1']
    
    p_x_y1 = (p_xj_y1 ** matrix).prod(axis=1) * p_y
    p_x_y0 = (p_xj_y0 ** matrix).prod(axis=1) * (1 - p_y)    
    predict = p_x_y1 / (p_x_y1 + p_x_y0)
    result = [1 if x > 0.5 else 0 for x in predict]
    return result
    
    


def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # Find five most indicative words of spam
    # Use the metric log(p(x_j = 1 | y = 1) / p(x_j = 1 | y = 0))
    # So for each word, compute the log ratio of the probabilities
    # Then sort the words by this ratio and return the top 5
    p_xj_y0 = model['p_xj_y0']
    p_xj_y1 = model['p_xj_y1']
    log_ratio = np.log(p_xj_y1 / p_xj_y0)
    top_five = np.argsort(log_ratio)[-5:][::-1]
    return [word for word, index in dictionary.items() if index in top_five]
    


def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    best_accuracy = 0
    for radius in radius_to_consider:
        svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius)
        # check accuracy of svm_predictions by comparing number of correct predictions to total number of predictions
        accuracy = np.sum(svm_predictions == val_labels)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_radius = radius
    return best_radius
    


train_messages, train_labels = util.load_spam_dataset('../data/ds6_train.tsv')
val_messages, val_labels = util.load_spam_dataset('../data/ds6_val.tsv')
test_messages, test_labels = util.load_spam_dataset('../data/ds6_test.tsv')

dictionary = create_dictionary(train_messages)

util.write_json('../output/p06_dictionary', dictionary)

train_matrix = transform_text(train_messages, dictionary)

np.savetxt('../output/p06_sample_train_matrix', train_matrix[:100,:])

val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

np.savetxt('../output/p06_naive_bayes_predictions', naive_bayes_predictions)

naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

util.write_json('../output/p06_top_indicative_words', top_5_words)

optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])

util.write_json('../output/p06_optimal_radius', optimal_radius)

print('The optimal SVM radius was {}'.format(optimal_radius))

svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)

svm_accuracy = np.mean(svm_predictions == test_labels)

print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))


Naive Bayes had an accuracy of 0.953405017921147 on the testing set
The top 5 indicative words for Naive Bayes are:  ['&', 'cash', 'reply', 'text', 'txt', 'win']
The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9713261648745519 on the testing set
